## Introduction à la quantization 

Laurent cetinsoy

Les réseaux de neurones prennent beaucoup de place et il peut être difficile de les faire rentrer sur certains dispositifs embarqués. 

Il existe plusieurs méthodes pour réduire la taille et augmenter la vitesse d'executer des réseaux de neurone. Par exemple il y a ce qu'on appelle la quantization et le pruning.

Dans ce notebook on va faire une introduction à la quantization avec la librairie tensorflow lite.


## Quantization post training

Dans un premier temps on va quantifier notre réseau après l'avoir entraîné normalement. 


Entraîner un réseau de neurone convolutionnel simple avec keras pour faire de la classification MNIST (ou un autre dataset simple de votre choix si (vous en avez marre de ce dataset - https://keras.io/api/datasets/)




In [25]:
import tensorflow as tf
import numpy as np
import tensorflow.keras as keras
from tensorflow.keras import layers
 
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

# Scale images to the [0, 1] range
x_train = x_train.astype("float32") / 255
x_test = x_test.astype("float32") / 255
# Make sure images have shape (28, 28, 1)
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)

num_classes = 10
input_shape = (28, 28, 1)

y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)



model = keras.Sequential(
    [
        keras.Input(shape=input_shape),
        layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation="softmax"),
    ]
)

model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

model.fit(x_train, y_train, batch_size=128, epochs=5, validation_split=0.1)

Epoch 1/5


2022-11-30 16:05:24.457143: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 169344000 exceeds 10% of free system memory.


422/422 [==============================] - 30s 68ms/step - loss: 0.3624 - accuracy: 0.8908 - val_loss: 0.0870 - val_accuracy: 0.9745
Epoch 2/5
422/422 [==============================] - 29s 68ms/step - loss: 0.1074 - accuracy: 0.9672 - val_loss: 0.0541 - val_accuracy: 0.9843
Epoch 3/5
422/422 [==============================] - 28s 67ms/step - loss: 0.0818 - accuracy: 0.9741 - val_loss: 0.0441 - val_accuracy: 0.9878
Epoch 4/5
422/422 [==============================] - 29s 70ms/step - loss: 0.0693 - accuracy: 0.9784 - val_loss: 0.0401 - val_accuracy: 0.9883
Epoch 5/5
422/422 [==============================] - 29s 68ms/step - loss: 0.0604 - accuracy: 0.9810 - val_loss: 0.0382 - val_accuracy: 0.9898


Afficher le nombre de paramètre du modèle

In [26]:
model.summary()

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_16 (Conv2D)          (None, 26, 26, 32)        320       
                                                                 
 max_pooling2d_16 (MaxPoolin  (None, 13, 13, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_17 (Conv2D)          (None, 11, 11, 64)        18496     
                                                                 
 max_pooling2d_17 (MaxPoolin  (None, 5, 5, 64)         0         
 g2D)                                                            
                                                                 
 flatten_8 (Flatten)         (None, 1600)              0         
                                                                 
 dropout_8 (Dropout)         (None, 1600)             

Sauvegarder votre modèle et afficher la taille du fichier. Si on applique une bête règle de trois, quelle est la taille occupée par paramètre ? 

In [27]:
!mkdir -p saved_model
saved_model_dir = "saved_model"

model.save('saved_model/model_1')

import os

print("Taille du fichier contenant le modèle:", os.path.getsize('saved_model/model_1'))

INFO:tensorflow:Assets written to: saved_model/model_1/assets


INFO:tensorflow:Assets written to: saved_model/model_1/assets


Taille du fichier contenant le modèle: 4096


On va maintenant convertir notre modèle keras en modèle tensorflow lite. 

Installer la librairie tensorflow lite créer une instance de la class TFLiteConverter à partir de votre modèle keras


In [28]:
%pip install tflite

import tflite

converter = tf.lite.TFLiteConverter.from_keras_model(model)

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


Convertir votre modèle et le sauvegarder dans un fichier nommé model.tflite. Sa taille est-elle plus petite ? 

In [29]:
tflite_model = converter.convert()

open("saved_model/model.tflite", "wb").write(tflite_model)
print("Taille du fichier contenant le modèle:", os.path.getsize('saved_model/model.tflite'))

INFO:tensorflow:Assets written to: /tmp/tmpi93lm8qj/assets


INFO:tensorflow:Assets written to: /tmp/tmpi93lm8qj/assets


Taille du fichier contenant le modèle: 142360


2022-11-30 16:08:25.336561: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:362] Ignored output_format.
2022-11-30 16:08:25.336669: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:365] Ignored drop_control_dependency.
2022-11-30 16:08:25.337077: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /tmp/tmpi93lm8qj
2022-11-30 16:08:25.340058: I tensorflow/cc/saved_model/reader.cc:89] Reading meta graph with tags { serve }
2022-11-30 16:08:25.340253: I tensorflow/cc/saved_model/reader.cc:130] Reading SavedModel debug info (if present) from: /tmp/tmpi93lm8qj
2022-11-30 16:08:25.359392: I tensorflow/cc/saved_model/loader.cc:229] Restoring SavedModel bundle.
2022-11-30 16:08:25.471254: I tensorflow/cc/saved_model/loader.cc:213] Running initialization op on SavedModel bundle at path: /tmp/tmpi93lm8qj
2022-11-30 16:08:25.514444: I tensorflow/cc/saved_model/loader.cc:305] SavedModel load for tags { serve }; Status: success: OK. Took 177371 

La taille est plus grande.

On va maintenant spécifier des optimisations au converter. 

1. Recréer un converter

2. modifier son attribut optimizations pour ajouter une liste d'optimisation avec la valeur tf.lite.Optimize.DEFAULT

3. Relancer la conversion du modèle, sauvegarder le modèle et regarder la taille du fichier généré

In [30]:
converter_2 = tf.lite.TFLiteConverter.from_keras_model(model)

converter_2.optimizations = [tf.lite.Optimize.DEFAULT]

tflite_model_2 = converter_2.convert()

open("saved_model/model_2.tflite", "wb").write(tflite_model_2)
print("Taille du fichier contenant le modèle:", os.path.getsize('saved_model/model_2.tflite'))

INFO:tensorflow:Assets written to: /tmp/tmpbfu8wh87/assets


INFO:tensorflow:Assets written to: /tmp/tmpbfu8wh87/assets


Taille du fichier contenant le modèle: 39944


2022-11-30 16:08:35.648852: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:362] Ignored output_format.
2022-11-30 16:08:35.648889: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:365] Ignored drop_control_dependency.
2022-11-30 16:08:35.649007: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /tmp/tmpbfu8wh87
2022-11-30 16:08:35.650890: I tensorflow/cc/saved_model/reader.cc:89] Reading meta graph with tags { serve }
2022-11-30 16:08:35.650967: I tensorflow/cc/saved_model/reader.cc:130] Reading SavedModel debug info (if present) from: /tmp/tmpbfu8wh87
2022-11-30 16:08:35.657614: I tensorflow/cc/saved_model/loader.cc:229] Restoring SavedModel bundle.
2022-11-30 16:08:35.705922: I tensorflow/cc/saved_model/loader.cc:213] Running initialization op on SavedModel bundle at path: /tmp/tmpbfu8wh87
2022-11-30 16:08:35.729019: I tensorflow/cc/saved_model/loader.cc:305] SavedModel load for tags { serve }; Status: success: OK. Took 80012 m

Quelle type  de quantization Optimize.Default, utilise-t-elle ?



La quantization par défaut est la post-training.


## Quantization aware training 

Dans cette section on va s'intéresser à l'entraînement sensible à la quantification. L'idée est de simuler les effets de la quantification pendant l'entraînement pour que le modèle ajuste les poids afin de tenir ocmpte de la quantification. 

Reprendre le modèle entraîné sur MNIST


In [48]:
model = keras.Sequential(
    [
        keras.Input(shape=input_shape),
        layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation="softmax"),
    ]
)

model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

A l'aide de la fonction quantize de tensorflow_model_optimization, créer une seconde version de votre modèle entraîné nommé qat_model

In [49]:
import tensorflow_model_optimization as tfmot

qat_model = tfmot.quantization.keras.quantize_model(model)
qat_model

Compiler le modèle

In [50]:
qat_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

Afficher le summury du modèle. D'après vous ce modèle est-il quantifié ? 

In [51]:
qat_model.summary()
# On voit que les layers sont toutes mises comme quantizeLayer donc le modèle est quantifié

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 quantize_layer_5 (QuantizeL  (None, 28, 28, 1)        3         
 ayer)                                                           
                                                                 
 quant_conv2d_18 (QuantizeWr  (None, 26, 26, 32)       387       
 apperV2)                                                        
                                                                 
 quant_max_pooling2d_18 (Qua  (None, 13, 13, 32)       1         
 ntizeWrapperV2)                                                 
                                                                 
 quant_conv2d_19 (QuantizeWr  (None, 11, 11, 64)       18627     
 apperV2)                                                        
                                                                 
 quant_max_pooling2d_19 (Qua  (None, 5, 5, 64)        

Réentraîner votre modèle sur un sous ensemble des modèles sur une ou deux epochs et afficher la performance sur le train et test set

In [52]:
# train model on subset of models
qat_model.fit(x_train, y_train, batch_size=128, epochs=2, validation_split=0.1)

Epoch 1/2
422/422 [==============================] - 44s 102ms/step - loss: 0.3816 - accuracy: 0.8835 - val_loss: 0.0811 - val_accuracy: 0.9785
Epoch 2/2
422/422 [==============================] - 37s 87ms/step - loss: 0.1127 - accuracy: 0.9652 - val_loss: 0.0536 - val_accuracy: 0.9857


In [41]:
# display performance of model
qat_model.evaluate(x_test, y_test, verbose=1)

313/313 [==============================] - 4s 14ms/step - loss: 0.0285 - accuracy: 0.9899


[0.02853892557322979, 0.9898999929428101]

Convertir votre modèle avec TFLite

In [53]:
converter_qat = tf.lite.TFLiteConverter.from_keras_model(qat_model)
converter_qat.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_qat_model_2 = converter_qat.convert()

INFO:tensorflow:Assets written to: /tmp/tmpmws1z2rt/assets


INFO:tensorflow:Assets written to: /tmp/tmpmws1z2rt/assets
/home/unikarah/.local/lib/python3.10/site-packages/tensorflow/lite/python/convert.py:765: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn("Statistics for quantized inputs were expected, but not "
2022-11-30 16:25:38.339623: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:362] Ignored output_format.
2022-11-30 16:25:38.339651: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:365] Ignored drop_control_dependency.
2022-11-30 16:25:38.339871: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /tmp/tmpmws1z2rt
2022-11-30 16:25:38.347332: I tensorflow/cc/saved_model/reader.cc:89] Reading meta graph with tags { serve }
2022-11-30 16:25:38.347364: I tensorflow/cc/saved_model/reader.cc:130] Reading SavedModel debug info (if present) from: /tmp/tmpmws1z2rt
2022-11-30 16:25:38.386978: I tensorflow/cc/saved_model/loader.

AttributeError: 'bytes' object has no attribute 'save'

Comparer la performance du modèle Quantified aware training, au modèle original et au modèle quantifié post training

In [47]:
tflite_qat_model_2
# FIXME: pb de loading

AttributeError: 'bytes' object has no attribute 'evaluate_tflite'

Sauvegarder le modèle QAT et comparer les tailles des modèles

In [56]:

open("saved_model/tflite_qat_model_2.tflite", "wb").write(tflite_qat_model_2)

print("Taille du fichier contenant le modèle:", os.path.getsize('saved_model/tflite_qat_model_2.tflite'))

Taille du fichier contenant le modèle: 41896


Bonus : déployer votre modèle sur votre téléphone ou un dispositif embarqué si vous en disposez d'un. 

Bonus : Obtenir un modèle qui sera à la fois quantifié et élagué (prunned) en s'aidant de la documentation (https://www.tensorflow.org/model_optimization/guide/pruning/pruning_with_keras)

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=0d51e245-899d-41d6-b23b-cf3e4bbbc6ea' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>